# import packages

In [1]:
import os
import torch
import pandas as pd
from transformers import BertModel, DistilBertModel
# import kobert_tokenizer

In [2]:
os.getcwd()

'C:\\Users\\Administrator\\Desktop\\Work\\2. 인공지능리서치AIR\\1. source code'

## save pre-trained kobert model (2021.07.30)

In [6]:
# torch.save(model, 'pretrained_kobert_20210730.pth')

# data sets

In [3]:
data_path = "C:/Users/Administrator/Desktop/Work/2. 인공지능리서치AIR/2. data sets/1. input data"
file_name1 = "(1차)13,500건.xlsx"
os.path.join(data_path, file_name1)
data_set1 = pd.read_excel(os.path.join(data_path, file_name1))

In [4]:
data_set1.shape

(13500, 9)

In [5]:
file_name2 = "(2차)34,500건.xlsx"
os.path.join(data_path, file_name2)
data_set2 = pd.read_excel(os.path.join(data_path, file_name2))

In [6]:
data_set2.shape

(34500, 9)

In [7]:
data_set = data_set1.append(data_set2)

In [8]:
data_set.shape

(48000, 9)

In [9]:
data_set.columns = ['uid',
                   'publisher',
                   'title',
                   'summary',
                   'content',
                   'content_url',
                    'update_at',
                   'importance',
                   'polarity']

In [10]:
data_set.head(1)

,uid,publisher,title,summary,content,content_url,update_at,importance,polarity
0,354652739944452421,뉴스1,김진석 CJ헬로비전 대표 '주주총회 70%이상 찬성으로 합병안 가결',임세영 기자 = 김진석 CJ헬로비전 대표가 26일 오전 서울 마포구 상암동 누리꿈스...,임세영 기자 = 김진석 CJ헬로비전 대표가 26일 오전 서울 마포구 상암동 누리꿈스...,http://news.naver.com/main/read.nhn?mode=LSD&m...,2016-02-26T10:25:00,+,+


In [128]:
data_set_cleaned1 = data_set[data_set.importance.notnull()]
# data_set_cleaned1.shape
data_set_cleaned1_2 = data_set_cleaned1[data_set_cleaned1.polarity.notnull()]
data_set_cleaned2 = data_set_cleaned1_2[data_set_cleaned1_2.importance != '0']
# data_set_cleaned2.shape
data_set_cleaned2['importance'].value_counts()

-    23854
+    14656
Name: importance, dtype: int64

In [125]:
data_set_cleaned2.head(1)

,uid,publisher,title,summary,content,content_url,update_at,importance,polarity
0,354652739944452421,뉴스1,김진석 CJ헬로비전 대표 '주주총회 70%이상 찬성으로 합병안 가결',임세영 기자 = 김진석 CJ헬로비전 대표가 26일 오전 서울 마포구 상암동 누리꿈스...,임세영 기자 = 김진석 CJ헬로비전 대표가 26일 오전 서울 마포구 상암동 누리꿈스...,http://news.naver.com/main/read.nhn?mode=LSD&m...,2016-02-26T10:25:00,+,+


In [13]:
from sklearn.preprocessing import LabelEncoder

In [129]:
encoder = LabelEncoder()

# importance
encoder.fit(data_set_cleaned2['importance'])
data_set_cleaned2['importance_int'] = encoder.transform(data_set_cleaned2['importance'])
# polarity
encoder.fit(data_set_cleaned2['polarity'])
data_set_cleaned2['polarity_int'] = encoder.transform(data_set_cleaned2['polarity'])

<ipython-input-129-7b66ed1a4174>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_set_cleaned2['importance_int'] = encoder.transform(data_set_cleaned2['importance'])
<ipython-input-129-7b66ed1a4174>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_set_cleaned2['polarity_int'] = encoder.transform(data_set_cleaned2['polarity'])


In [130]:
mapping = dict(zip(range(len(encoder.classes_)), encoder.classes_))
mapping

{0: '+', 1: '-', 2: '0'}

In [131]:
data_set_cleaned2['importance'].value_counts()

-    23854
+    14656
Name: importance, dtype: int64

In [171]:
data_set_cleaned2['importance_int']=data_set_cleaned2['importance_int'].astype(np.int64)

<ipython-input-171-88e4bed54ae8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_set_cleaned2['importance_int']=data_set_cleaned2['importance_int'].astype(np.int64)


In [172]:
data_set_cleaned2['importance_int']

0        0
2        1
3        1
4        1
7        0
        ..
34495    1
34496    1
34497    1
34498    1
34499    1
Name: importance_int, Length: 38510, dtype: int64

In [132]:
data_set_cleaned2['importance_int'].value_counts()

1    23854
0    14656
Name: importance_int, dtype: int64

In [18]:
data_set_cleaned2['polarity'].value_counts()

0    25776
+    14702
-     6751
Name: polarity, dtype: int64

In [19]:
data_set_cleaned2['polarity_int'].value_counts()

2    25776
0    14702
1     6751
Name: polarity_int, dtype: int64

# Classifier 3

### https://zzaebok.github.io/deep_learning/nlp/Bert-for-classification/

In [20]:
# pip install pytorch-transformers

In [175]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from pytorch_transformers import BertTokenizer, BertForSequenceClassification, BertConfig
from torch.optim import Adam
import torch.nn.functional as F

In [176]:
data_set_cleaned2.head(1)

,uid,publisher,title,summary,content,content_url,update_at,importance,polarity,importance_int,polarity_int
0,354652739944452421,뉴스1,김진석 CJ헬로비전 대표 '주주총회 70%이상 찬성으로 합병안 가결',임세영 기자 = 김진석 CJ헬로비전 대표가 26일 오전 서울 마포구 상암동 누리꿈스...,임세영 기자 = 김진석 CJ헬로비전 대표가 26일 오전 서울 마포구 상암동 누리꿈스...,http://news.naver.com/main/read.nhn?mode=LSD&m...,2016-02-26T10:25:00,+,+,0,0


In [177]:
new_data = data_set_cleaned2[['content', 'importance_int']]

In [178]:
new_data.shape

(38510, 2)

In [179]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(new_data, test_size=0.2, random_state=42)
print("train shape is:", len(train))
print("test shape is:", len(test))

train shape is: 30808
test shape is: 7702


In [180]:
class AirDataset(Dataset):
    ''' Dataset '''
    def __init__(self, df):
        self.df = df

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        text = self.df.iloc[idx, 0]
        label = self.df.iloc[idx, 1]
        return text, label

In [192]:
Air_train_dataset = AirDataset(train[:20])
train_loader = DataLoader(Air_train_dataset,
                         batch_size = 10,
                         shuffle = True,
                         num_workers = 0)

In [193]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased')

In [194]:
optimizer = Adam(model.parameters(), lr=1e-6)

itr = 500
p_itr = 500
epochs = 1
total_loss = 0
total_len = 0
total_correct = 0


model.train()
for epoch in range(epochs):
    
    for text, label in train_loader:
        optimizer.zero_grad()
        
        # encoding and zero padding
        encoded_list = [tokenizer.encode(t, add_special_tokens=True) for t in text]
        # padded_list =  [e + [0] * (119547-len(e)) for e in encoded_list]
        padded_list =  [e[:min(len(e), 512)] + [0] * (512-len(e[:min(512, len(e))])) for e in encoded_list]
        
        sample = torch.tensor(padded_list)
        # sample, label = sample.to(device), label.to(device)
        # sample, label = sample.to(device), label.to(device)
        labels = torch.tensor(label)
        outputs = model(sample, labels=labels)
        loss, logits = outputs

        pred = torch.argmax(F.softmax(logits), dim=1)
        correct = pred.eq(labels)
        total_correct += correct.sum().item()
        total_len += len(labels)
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
        
        if itr % p_itr == 0:
            print('[Epoch {}/{}] Iteration {} -> Train Loss: {:.4f}, Accuracy: {:.3f}'.format(epoch+1, epochs, itr, total_loss/p_itr, total_correct/total_len))
            total_loss = 0
            total_len = 0
            total_correct = 0

        itr+=1

Token indices sequence length is longer than the specified maximum sequence length for this model (861 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1948 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (669 > 512). Running this sequence through the model will result in indexing errors
<ipython-input-194-8f6d013fa936>:25: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(label)
<ipython-input-194-8f6d013fa936>:29: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  pred = torch.argmax(F.softmax

[Epoch 1/1] Iteration 500 -> Train Loss: 0.0014, Accuracy: 0.400


In [ ]:
# evaluation
model.eval()

nsmc_eval_dataset = NsmcDataset(test_df)
eval_loader = DataLoader(nsmc_eval_dataset, batch_size=2, shuffle=False, num_workers=2)

total_loss = 0
total_len = 0
total_correct = 0

for text, label in eval_loader:
    encoded_list = [tokenizer.encode(t, add_special_tokens=True) for t in text]
    padded_list =  [e + [0] * (512-len(e)) for e in encoded_list]
    sample = torch.tensor(padded_list)
    sample, label = sample.to(device), label.to(device)
    labels = torch.tensor(label)
    outputs = model(sample, labels=labels)
    _, logits = outputs

    pred = torch.argmax(F.softmax(logits), dim=1)
    correct = pred.eq(labels)
    total_correct += correct.sum().item()
    total_len += len(labels)

print('Test accuracy: ', total_correct / total_len)